In [1]:
## First we will see the example

In [1]:
import torch
import torch.nn as nn

# get device
device = 'cpu' #if not torch.cuda.is_available() else 'cuda'

# a simple NN
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.layer1 = nn.Linear(in_features=4, 
                                out_features=2)
    def forward(self,x):
        x = self.layer1(x)
        return x
    

# Transform the model to device
net = NN().to(device)

In [3]:
# freeze the weights and discard the gradients
for parm in net.parameters():
    parm.requires_grad=False


In [4]:
# create a random input, a input is mandatory for IR.
x = torch.randn(1,4, device=device)

# Use torch script to get IR
trace_nn = torch.jit.trace(net, x)

print(trace_nn.graph) # To visualize the IR

graph(%self.1 : __torch__.NN,
      %input : Float(1:4, 4:1, requires_grad=0, device=cpu)):
  %14 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="layer1"](%self.1)
  %16 : Tensor = prim::CallMethod[name="forward"](%14, %input)
  return (%16)



In [5]:
x = torch.randn(1,4, device=device)
trace_nn(x)

tensor([[ 0.6583, -0.5894]])

In [6]:
x = torch.randn(1,4, device=device)
trace_nn(x)

tensor([[ 0.0036, -1.0990]])

### Importing my trained model

In [2]:
from detector import Detector

In [3]:
def load_model(model_path, num_classes, device='cpu'):
    model = Detector(num_classes)
    model.load_state_dict(
                    torch.load(model_path)
                )
    model = model.eval()
    model.to(device)
    return model

In [4]:
model_path = "/media/rahul/a079ceb2-fd12-43c5-b844-a832f31d5a39/Projects/autonomous_cars/Object_Detector_for_road/SSD_Detector_for_road_training/checkpoints/Detector_best.pth"
num_classes = 12
device = 'cpu'

In [5]:
model = load_model(model_path, num_classes, device)

In [6]:
model

Detector(
  (fpn): FPN(
    (feature_extractor): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchN

In [10]:
# freeze the weights and discard the gradients
for parm in model.parameters():
    parm.requires_grad=False

In [11]:
x = torch.randn(1,3,300,300, device=device)

In [12]:
x.shape

torch.Size([1, 3, 300, 300])

In [13]:
loc_preds, cls_preds = model(x)

In [14]:
# Use torch script to get IR
trace_nn = torch.jit.trace(model, x)

print(trace_nn.graph) # To visualize the IR

graph(%self.1 : __torch__.detector.___torch_mangle_133.Detector,
      %input.1 : Float(1:270000, 3:90000, 300:300, 300:1, requires_grad=0, device=cpu)):
  %2644 : __torch__.torch.nn.modules.container.___torch_mangle_132.Sequential = prim::GetAttr[name="cls_head"](%self.1)
  %2624 : __torch__.torch.nn.modules.container.___torch_mangle_122.Sequential = prim::GetAttr[name="loc_head"](%self.1)
  %2604 : __torch__.fpn.___torch_mangle_112.FPN = prim::GetAttr[name="fpn"](%self.1)
  %2995 : (Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor, Tensor) = prim::CallMethod[name="forward"](%2604, %input.1)
  %2987 : Float(1:92416, 64:1444, 38:38, 38:1, requires_grad=0, device=cpu), %2988 : Float(1:23104, 64:361, 19:19, 19:1, requires_grad=0, device=cpu), %2989 : Float(1:23104, 64:361, 19:19, 19:1, requires_grad=0, device=cpu), %2990 : Float(1:6400, 64:100, 10:10, 10:1, requires_grad=0, device=cpu), %2991 : Float(1:6400, 64:100, 10:10, 10:1, requires_grad=0, device=cpu), %2992 : Float(1:1600, 6

In [15]:
x = torch.randn(1,3,300,300, device=device)
trace_nn(x)

(tensor([[[ 8.4098e-03, -5.2960e-02, -2.7917e-01, -4.9883e-02],
          [-2.0193e-02, -5.8481e-02, -3.4454e-01,  1.1742e-01],
          [-5.3269e-02, -1.4594e-01, -3.5848e-01,  4.1374e-01],
          ...,
          [-2.2324e-04,  1.2489e-01, -1.9495e-01, -6.1979e-02],
          [ 1.4784e-02, -3.6648e-03, -3.6295e-01, -3.1864e-02],
          [ 1.6755e-04, -7.0247e-02, -3.0883e-01,  5.9969e-02]]]),
 tensor([[[20.4293,  0.3109, -0.0582,  ..., -2.9092,  0.3104, -6.0114],
          [19.0180,  0.6980,  0.6170,  ..., -4.4499, -3.2871, -4.9624],
          [18.9035,  0.6408,  0.9066,  ..., -2.7602, -2.9714, -3.3365],
          ...,
          [18.7619,  7.1949,  6.4355,  ..., -4.2550, -1.1850, -5.6571],
          [18.0575,  8.6073,  2.3610,  ..., -4.3963, -5.8178, -4.0444],
          [17.3651,  3.3296,  1.8229,  ..., -1.8929, -2.3507, -2.3172]]]))

In [16]:
# We can also save and load this scripted-model.
torch.jit.save(trace_nn, 'tiny_model.pt')

# Now we can load this model
reloaded_tiny_model = torch.jit.load('tiny_model.pt')

print("Output is ", reloaded_tiny_model(x))

Output is  (tensor([[[ 8.4098e-03, -5.2960e-02, -2.7917e-01, -4.9883e-02],
         [-2.0193e-02, -5.8481e-02, -3.4454e-01,  1.1742e-01],
         [-5.3269e-02, -1.4594e-01, -3.5848e-01,  4.1374e-01],
         ...,
         [-2.2324e-04,  1.2489e-01, -1.9495e-01, -6.1979e-02],
         [ 1.4784e-02, -3.6648e-03, -3.6295e-01, -3.1864e-02],
         [ 1.6755e-04, -7.0247e-02, -3.0883e-01,  5.9969e-02]]]), tensor([[[20.4293,  0.3109, -0.0582,  ..., -2.9092,  0.3104, -6.0114],
         [19.0180,  0.6980,  0.6170,  ..., -4.4499, -3.2871, -4.9624],
         [18.9035,  0.6408,  0.9066,  ..., -2.7602, -2.9714, -3.3365],
         ...,
         [18.7619,  7.1949,  6.4355,  ..., -4.2550, -1.1850, -5.6571],
         [18.0575,  8.6073,  2.3610,  ..., -4.3963, -5.8178, -4.0444],
         [17.3651,  3.3296,  1.8229,  ..., -1.8929, -2.3507, -2.3172]]]))


##### The trace function has some problems related to the loops, thus it is not recommended , instead we use the script function by JIT

In [40]:
model(x)

(tensor([[[ 0.0029, -0.0488, -0.3023, -0.0420],
          [-0.0523, -0.0612, -0.3806,  0.0822],
          [-0.0880, -0.1440, -0.3923,  0.3828],
          ...,
          [ 0.0174,  0.1108, -0.2049, -0.0773],
          [ 0.0306, -0.0204, -0.3729, -0.0147],
          [ 0.0100, -0.0903, -0.2930,  0.0934]]]),
 tensor([[[22.2652,  0.8254,  0.1599,  ..., -3.0745,  0.6601, -6.6966],
          [21.2056,  1.0720,  0.7180,  ..., -4.7390, -3.4844, -5.4035],
          [21.1097,  0.9526,  1.0791,  ..., -2.9896, -3.2018, -3.5883],
          ...,
          [18.8358,  8.4834,  6.0018,  ..., -4.3125, -0.9588, -5.8613],
          [17.7656,  9.6248,  2.0490,  ..., -4.3750, -5.6874, -3.8808],
          [17.2748,  4.2087,  1.5717,  ..., -1.8908, -2.6076, -2.3024]]]))

In [8]:


scripted_nn = torch.jit.script(model)




In [9]:
x = torch.randn(1,3,300,300, device=device)
scripted_nn(x)
# print(scripted_nn.graph_for(x))

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/media/rahul/a079ceb2-fd12-43c5-b844-a832f31d5a39/Projects/autonomous_cars/Object_Detector_for_road/SSD_Detector_for_road_training/SSD_Trainer/detector.py", line 18, in forward
    def forward(self, x):
        fms = self.fpn(x)
              ~~~~~~~~ <--- HERE
        loc_preds = []
        cls_preds = []
  File "/media/rahul/a079ceb2-fd12-43c5-b844-a832f31d5a39/Projects/autonomous_cars/Object_Detector_for_road/SSD_Detector_for_road_training/SSD_Trainer/fpn.py", line 92, in forward
        output.append(self.toplayer2(self._upsample_add(output[-1], self.latlayer3(layer2_output))))
        
        return output[::-1]
               ~~~~~~~~~~~~ <--- HERE
RuntimeError: vector::_M_range_check: __n (which is 18446744073709551615) >= this->size() (which is 5)


In [ ]:
print("Output is:\n ", scripted_nn(x))

In [44]:
# create a tiny feed-forward layer and set its paramters to be non-trainable 
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Linear(in_features=4, out_features=2)
        
    def forward(self,x):
        x = self.layer1(x)
        return x


# transform model to device
small_net = Network().to(device)

# freeze the model
for parm in small_net.parameters():
    parm.requires_grad=False
    
x = torch.ones(1, 4).to(device)

In [45]:
scripted_nn = torch.jit.script(small_net)
scripted_nn(x)
print(scripted_nn.graph_for(x))

print("Output is:\n ", scripted_nn(x))

graph(%self : __torch__.___torch_mangle_235.Network,
      %x.1 : Tensor):
  %3 : int = prim::Constant[value=2]() # /home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/nn/functional.py:1688:22
  %2 : int = prim::Constant[value=1]()
  %4 : __torch__.torch.nn.modules.linear.___torch_mangle_234.Linear = prim::GetAttr[name="layer1"](%self)
  %5 : Tensor = prim::GetAttr[name="weight"](%4)
  %6 : Tensor = prim::GetAttr[name="bias"](%4)
  %8 : int = aten::dim(%x.1) # /home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/nn/functional.py:1688:7
  %9 : bool = aten::eq(%8, %3) # /home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/nn/functional.py:1688:7
  %x.3 : Tensor = prim::If(%9) # /home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/nn/functional.py:1688:4
    block0():
      %12 : Tensor = aten::t(%5) # /home/rahul/.conda/envs/opcv/lib/python3.6/site-packages/torch/nn/functional.py:1690:39
      %15 : Tensor = aten::mm(%x.1, %12) # <string>:3:24
   

In [46]:
x.shape

torch.Size([1, 4])